In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_ref import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_e-4wd_Battery.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_id_75_110_Westinghouse.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: ref
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the lay

maximum steps, simulation is done ... 
Episode: 42 Total reward: -528.2218750089339 Explore P: 0.3224 SOC: 0.6632 Cumulative_SOC_deviation: 47.8839 Fuel Consumption: 49.3828
maximum steps, simulation is done ... 
Episode: 43 Total reward: -642.5577387442607 Explore P: 0.3140 SOC: 0.7052 Cumulative_SOC_deviation: 58.9803 Fuel Consumption: 52.7549
maximum steps, simulation is done ... 
Episode: 44 Total reward: -787.9204644377 Explore P: 0.3057 SOC: 0.7909 Cumulative_SOC_deviation: 72.8348 Fuel Consumption: 59.5722
maximum steps, simulation is done ... 
Episode: 45 Total reward: -814.9887254151138 Explore P: 0.2977 SOC: 0.6279 Cumulative_SOC_deviation: 76.7720 Fuel Consumption: 47.2685
maximum steps, simulation is done ... 
Episode: 46 Total reward: -656.0446445223959 Explore P: 0.2899 SOC: 0.6962 Cumulative_SOC_deviation: 60.3768 Fuel Consumption: 52.2770
maximum steps, simulation is done ... 
Episode: 47 Total reward: -543.756077861241 Explore P: 0.2824 SOC: 0.6281 Cumulative_SOC_devia

maximum steps, simulation is done ... 
Episode: 90 Total reward: -430.0466930697976 Explore P: 0.0936 SOC: 0.6557 Cumulative_SOC_deviation: 38.1269 Fuel Consumption: 48.7777
maximum steps, simulation is done ... 
Episode: 91 Total reward: -387.18139259652645 Explore P: 0.0914 SOC: 0.6271 Cumulative_SOC_deviation: 34.0273 Fuel Consumption: 46.9082
maximum steps, simulation is done ... 
Episode: 92 Total reward: -349.7712603122939 Explore P: 0.0892 SOC: 0.6303 Cumulative_SOC_deviation: 30.3048 Fuel Consumption: 46.7237
maximum steps, simulation is done ... 
Episode: 93 Total reward: -364.15955720243664 Explore P: 0.0870 SOC: 0.6265 Cumulative_SOC_deviation: 31.7927 Fuel Consumption: 46.2322
maximum steps, simulation is done ... 
Episode: 94 Total reward: -315.3530926931585 Explore P: 0.0849 SOC: 0.6231 Cumulative_SOC_deviation: 26.9079 Fuel Consumption: 46.2737
maximum steps, simulation is done ... 
Episode: 95 Total reward: -358.6432289332972 Explore P: 0.0829 SOC: 0.6102 Cumulative_SOC

maximum steps, simulation is done ... 
Episode: 137 Total reward: -383.79604581310656 Explore P: 0.0330 SOC: 0.6244 Cumulative_SOC_deviation: 33.7440 Fuel Consumption: 46.3563
maximum steps, simulation is done ... 
Episode: 138 Total reward: -316.0703467437764 Explore P: 0.0324 SOC: 0.6125 Cumulative_SOC_deviation: 27.0658 Fuel Consumption: 45.4122
maximum steps, simulation is done ... 
Episode: 139 Total reward: -326.9926645628037 Explore P: 0.0318 SOC: 0.6094 Cumulative_SOC_deviation: 28.1783 Fuel Consumption: 45.2100
maximum steps, simulation is done ... 
Episode: 140 Total reward: -336.1030203365167 Explore P: 0.0312 SOC: 0.6471 Cumulative_SOC_deviation: 28.8064 Fuel Consumption: 48.0393
maximum steps, simulation is done ... 
Episode: 141 Total reward: -677.6014285386021 Explore P: 0.0306 SOC: 0.7090 Cumulative_SOC_deviation: 62.4293 Fuel Consumption: 53.3083
maximum steps, simulation is done ... 
Episode: 142 Total reward: -305.45062577935863 Explore P: 0.0301 SOC: 0.6100 Cumulati

maximum steps, simulation is done ... 
Episode: 184 Total reward: -332.29133271669906 Explore P: 0.0163 SOC: 0.6094 Cumulative_SOC_deviation: 28.7137 Fuel Consumption: 45.1542
maximum steps, simulation is done ... 
Episode: 185 Total reward: -352.80210390702564 Explore P: 0.0162 SOC: 0.6238 Cumulative_SOC_deviation: 30.6205 Fuel Consumption: 46.5975
maximum steps, simulation is done ... 
Episode: 186 Total reward: -275.53671741378116 Explore P: 0.0160 SOC: 0.6140 Cumulative_SOC_deviation: 22.9659 Fuel Consumption: 45.8774
maximum steps, simulation is done ... 
Episode: 187 Total reward: -358.65707000289734 Explore P: 0.0158 SOC: 0.6086 Cumulative_SOC_deviation: 31.3335 Fuel Consumption: 45.3220
maximum steps, simulation is done ... 
Episode: 188 Total reward: -300.3624048361761 Explore P: 0.0157 SOC: 0.6217 Cumulative_SOC_deviation: 25.4073 Fuel Consumption: 46.2889
maximum steps, simulation is done ... 
Episode: 189 Total reward: -353.8306842678796 Explore P: 0.0155 SOC: 0.6104 Cumula

In [11]:
with open("DDQN3_norm.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data

FileNotFoundError: [Errno 2] No such file or directory: 'results/replay_memory_size_effect.pkl'